In [1]:
import autograd.numpy as np

from autograd import grad
from autograd.misc.optimizers import adam
from autograd.scipy.misc import logsumexp
from keras.datasets import mnist
from keras.utils import np_utils

/home/gc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Pre-processing

Read in the *MNIST database of handwritten digits* from [Keras](https://keras.io/datasets/#mnist-database-of-handwritten-digits).

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
X_train.shape

(60000, 28, 28)

Flatten images to vectors.

In [4]:
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

In [5]:
X_train.shape

(60000, 784)

Normalise to unit interval.

In [6]:
max_X = X_train.max()
X_train = X_train / max_X
X_test = X_test / max_X

One-hot encode `y` labels.

In [7]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [8]:
y_train[:5, :]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

# Modelling

Define a function to predict classes given weights and biases (as `parameters`) and the data `X`.

In [9]:
def predict(parameters, X):
    for W, b in parameters:
        y_hat = np.dot(X, W) + b
        X = np.tanh(y_hat)
    return y_hat - logsumexp(y_hat, axis=1, keepdims=True)

Define the loss function to minimise.

In [10]:
def loss(parameters, X, y):
    return -np.sum(y * predict(parameters, X))

Define a function to measure the accuracy of predictions made using `parameters`.

In [11]:
def accuracy(parameters, X, y):
    y = np.argmax(y, axis=1)
    y_hat = np.argmax(predict(parameters, X), axis=1)
    return np.mean(y_hat == y)

Define a function to randomly initialise weights and biases.

In [12]:
def random_init(sigma, sizes, prng=np.random.RandomState(seed=42)):
    return [(sigma * prng.randn(m, n), sigma * prng.randn(n))  # (weights, biases)
            for m, n in zip(sizes[:-1], sizes[1:])]

Define parameters for the neural network and optimisation algorithm.

In [13]:
layer_sizes = (X_train.shape[1], 200, 100, y_train.shape[1])
sigma = 0.1
batch_size = 256
n_epochs = 5
step_size = 0.001

Compute number of batches and define a function to generate batch indices given the iteration number `it`.

In [14]:
n_batches = int(np.ceil(len(X_train) / batch_size))

def batch_indices(it):
    idx = it % n_batches
    return slice(idx * batch_size, (idx + 1) * batch_size)

Define objective function and its gradient.

In [15]:
def objective(parameters, it):
    idx = batch_indices(it)
    return loss(parameters, X_train[idx], y_train[idx])

In [16]:
objective_grad = grad(objective)

Initialise parameters randomly.

In [17]:
parameters = random_init(sigma, layer_sizes)

Check initial predictions and loss for the first batch.

In [18]:
predict(parameters, X_train[:5, :])

array([[-2.70023786, -2.44684557, -2.14167641, -1.54998636, -1.96201996,
        -1.82885366, -2.5333151 , -2.5707044 , -3.64097556, -3.40339709],
       [-2.5604389 , -2.84849808, -2.80874001, -1.50753379, -1.87884739,
        -2.01618161, -2.70002861, -2.06754373, -3.65626445, -2.55613636],
       [-2.54389909, -2.81593475, -2.4844252 , -2.43473089, -1.82139518,
        -2.34067401, -2.58995902, -2.2891326 , -2.14069762, -1.97627763],
       [-2.15704758, -2.85468154, -2.0956475 , -2.62614572, -2.07549301,
        -2.10108898, -3.29443049, -1.94373585, -2.4252526 , -2.16385303],
       [-2.70961953, -2.36461903, -2.5948118 , -2.05592123, -2.07706406,
        -1.60714808, -2.26015076, -2.76119053, -2.79174911, -2.49863842]])

In [19]:
objective(parameters, 0)

593.5535733141162

Optimise using [Adam](https://arxiv.org/abs/1412.6980).

In [20]:
print('Iteration | Training accuracy | Test accuracy')
def print_accuracy(parameters, it, gradient):
    if it % n_batches == 0:
        train_accuracy = accuracy(parameters, X_train, y_train)
        test_accuracy = accuracy(parameters, X_test, y_test)
        print('{:9} | {:16.2f}% | {:12.2f}%'.format(it // n_batches,
                                                    train_accuracy * 100,
                                                    test_accuracy * 100))

opt_parameters = adam(
    grad=objective_grad,
    x0=parameters,
    callback=print_accuracy,
    num_iters=n_epochs * n_batches,
    step_size=step_size
)

Iteration | Training accuracy | Test accuracy
        0 |             9.93% |         9.62%
        1 |            94.01% |        93.99%
        2 |            96.08% |        95.63%
        3 |            97.19% |        96.50%
        4 |            97.82% |        96.92%
